In [ ]:
import os
import sys
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
%matplotlib inline
sns.set_theme()

In [ ]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from src.strategy.model import Model
from src.strategy.environment import Environment
from src.strategy.agent import Agent
from src.strategy.buffer import Buffer
from src.utils import get_config, read_file
config = get_config.read_yaml()

In [ ]:
print("Starting Training...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
MODEL_PATH = config['paths']['model_directory']
CAPITAL = config['strategy']['capital']
SYMBOLS = config['data']['symbols']

hp = config['hyperparameters']
NUM_ASSETS = hp['num_assets']
INPUT_DIM = hp['input_dim']
ACTION_DIM = hp['action_dim']
NUM_LSTM_LAYERS = hp['num_lstm_layers']
HIDDEN_STATE_DIM = hp['hidden_state_dim']
ACTOR_HIDDEN_DIM = hp['actor_hidden_dim']
CRITIC_HIDDEN_DIM = hp['critic_hidden_dim']
GAMMA = hp['gamma']
GAE_LAMBDA = hp['gae_lambda']
CLIP_EPSILON = hp['clip_epsilon']
VALUE_LOSS_COEF = hp['value_loss_coef']
ENTROPY_LOSS_COEF = hp['entropy_loss_coef']
LEARNING_RATE = hp['learning_rate']
NUM_EPOCHS = hp['num_epochs']
ROLLOUT_SIZE = hp['rollout_size']
MINI_BATCH_SIZE = hp['mini_batch_size']
SEQUENCE_LENGTH = hp['seq_len']
train_data_norm = read_file.read_merged_training_data()
train_data_unnorm = read_file.read_merged_training_data(False)
print(f'MODEL_PATH: {MODEL_PATH}')
print(f'CAPITAL: {CAPITAL}')
print(f'SYMBOLS: {SYMBOLS}')
print(f'NUM_ASSETS: {NUM_ASSETS}')
print(f'INPUT_DIM: {INPUT_DIM}')
print(f'ACTION_DIM: {ACTION_DIM}')
print(f'NUM_LSTM_LAYERS: {NUM_LSTM_LAYERS}')
print(f'HIDDEN_STATE_DIM: {HIDDEN_STATE_DIM}')
print(f'ACTOR_HIDDEN_DIM: {ACTOR_HIDDEN_DIM}')
print(f'CRITIC_HIDDEN_DIM: {CRITIC_HIDDEN_DIM}')
print(f'GAMMA: {GAMMA}')
print(f'GAE_LAMBDA: {GAE_LAMBDA}')
print(f'CLIP_EPSILON: {CLIP_EPSILON}')
print(f'VALUE_LOSS_COEF: {VALUE_LOSS_COEF}')
print(f'ENTROPY_LOSS_COEF: {ENTROPY_LOSS_COEF}')
print(f'LEARNING_RATE: {LEARNING_RATE}')
print(f'NUM_EPOCHS: {NUM_EPOCHS}')
print(f'ROLLOUT_SIZE: {ROLLOUT_SIZE}')
print(f'BATCH_SIZE: {MINI_BATCH_SIZE}')
print(f'SEQUENCE_LENGTH: {SEQUENCE_LENGTH}')
print(train_data_norm)
print(train_data_unnorm)

In [ ]:
model = Model(INPUT_DIM,
              HIDDEN_STATE_DIM,
              NUM_ASSETS,
              NUM_LSTM_LAYERS,
              ACTOR_HIDDEN_DIM,
              CRITIC_HIDDEN_DIM,)
model

In [ ]:
agent = Agent(model,
              GAMMA,
              GAE_LAMBDA,
              CLIP_EPSILON,
              VALUE_LOSS_COEF,
              ENTROPY_LOSS_COEF,
              LEARNING_RATE,
              MINI_BATCH_SIZE,
              device,
              MODEL_PATH)
agent

In [ ]:
env = Environment(train_data_unnorm,
                  SEQUENCE_LENGTH,
                  NUM_ASSETS,
                  SYMBOLS,
                  CAPITAL)
env

In [ ]:
env.reset()

In [ ]:
buffer = Buffer(ROLLOUT_SIZE,
                SEQUENCE_LENGTH,
                INPUT_DIM,
                ACTION_DIM,
                device)
buffer.display()

In [ ]:
train_data_norm.iloc[0:SEQUENCE_LENGTH]

In [ ]:
buffer.store_state(train_data_norm.iloc[0:SEQUENCE_LENGTH].values)
buffer.states

In [ ]:
for step in tqdm(range(5)):
    buffer = agent.get_action_and_value(buffer)
    buffer = env.step(buffer.actions[buffer.current_step_action - 1], buffer)
# print(buffer.rewards.shape)
# print(buffer.values.shape)
# print(buffer.dones.shape)
buffer = agent.compute_gae(buffer=buffer, next_value=torch.tensor([0.0]), next_done=torch.tensor([0.0]))
buffer.actions

In [ ]:
buffer.advantages